In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense

In [ ]:
# Loading Credit Card Fraud Dataset:

url_inv_train = 'https://github.com/manugaco/ML_User_Cases/raw/master/Datasets/invoice_train.csv.zip?raw=true'
zip_inv_train = ZipFile(BytesIO(requests.get(url_inv_train).content), 'r')
inv_train = pd.read_csv(zip_inv_train.open(zip_inv_train.namelist()[0]), low_memory=False)